In [3]:
import pandas as pd
import openpyxl
import excel
import datetime as dt

In [6]:
#Open the workbook to read cell values
# The files is automatically closed again adter loading the data
book = openpyxl.load_workbook('../xl/stores.xlsx',data_only=True)

In [7]:
# Get a worksheet object by name or index (0-based)
sheet = book['2019']
sheet = book.worksheets[0]

In [8]:
book.sheetnames

['2019', '2020', '2019-2020']

In [9]:
for i in book.worksheets:
    print(i.title)

2019
2020
2019-2020


In [11]:
#Getting the dimensions,
# i.e., the used range of the sheet
sheet.max_row, sheet.max_column

(8, 6)

In [13]:
# Read the value of a single cell
# using 'A1' notation and using cell indices (1-based)
sheet['B6'].value
sheet.cell(row=6, column=2).value

'Boston'

In [14]:
# Read in a range of cell valuesby using our excel module
data = excel.read(book['2019'],(2,2),(8,6))
data[:2]

[['Store', 'Employees', 'Manager', 'Since', 'Flagship'],
 ['New York', 10, 'Sarah', datetime.datetime(2018, 7, 20, 0, 0), False]]

In [15]:
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.chart import BarChart, Reference
from openpyxl.styles import Font, colors
from openpyxl.styles.borders import Border, Side
from openpyxl.styles.alignment import Alignment
from openpyxl.styles.fills import PatternFill
import excel

In [16]:
# Instantiate a workbook
book = openpyxl.Workbook()